In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 22.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [20, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_a = pd.read_excel('data/부동산데이터분석_아파트단지별정보_서울_변수TF_13_변수작업완료.xlsx')
df_a

,법정동주소,단지명,단지코드,도로명주소,시군구,동리,거래허가제TF,아파트TF,주상복합TF,도시형주택TF,...,지역난방TF,복도식TF,혼합형TF,계단식TF,메이저시공사TF,조합시행TF,철근철골구조TF,홈네트워크TF,관리시설수,부대복리시설합
0,서울특별시 마포구 아현동 799 공덕자이(임대),공덕자이(임대),A10027951,서울특별시 마포구 마포대로26길 22,마포구,아현동,0,1,0,0,...,0,1,0,0,1,1,0,1,5,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
1,서울특별시 성북구 길음동 1285-5 길음뉴타운7단지제2,길음뉴타운7단지제2,A13611012,서울특별시 성북구 길음로13길 22,성북구,길음동,0,1,0,0,...,0,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
2,서울특별시 성북구 길음동 1284-9 길음뉴타운8단지제2,길음뉴타운8단지제2,A13611010,서울특별시 성북구 길음로 33,성북구,길음동,0,1,0,0,...,0,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 주민공동시설, 어린이놀이터"
3,서울특별시 마포구 도화동 564 마포동원베네스트,마포동원베네스트,A12170401,서울특별시 마포구 도화4길 41,마포구,도화동,0,1,0,0,...,0,0,0,1,0,0,0,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
4,서울특별시 강남구 청담동 23 청담2차현대아파트,청담2차현대아파트,A13510202,서울특별시 강남구 선릉로 748,강남구,청담동,1,1,0,0,...,1,0,0,1,1,0,0,0,2,"관리사무소, 어린이놀이터"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,서울특별시 강동구 천호동 46 강동이루미,강동이루미,A13402004,서울특별시 강동구 양재대로123길 30,강동구,천호동,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN
2795,서울특별시 용산구 한남동 726-74 남산맨션,남산맨션,A54016001,NaN,용산구,한남동,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN
2796,서울특별시 구로구 항동 1-5 그린빌라,그린빌라,A15271602,서울특별시 구로구 연동로 287,구로구,항동,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN
2797,서울특별시 강서구 화곡동 1156 우장산휴먼빌,우장산휴먼빌,A15701005,NaN,강서구,화곡동,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN


In [3]:
df_b = pd.read_excel('data/ID있는버전.xlsx')
df_b

,ID,시군구,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,거래유형
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,201801,3,130000,4,1987,언주로 3,-
1,서울특별시 동작구 신대방동 롯데관악타워 150,서울특별시 동작구 신대방동,롯데관악타워,150,201801,4,74000,27,1996,보라매로5길 51,-
2,서울특별시 동작구 신대방동 경남교수 80,서울특별시 동작구 신대방동,경남교수,80,201801,25,65000,17,2001,여의대방로10길 14,-
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,80,201801,27,85800,15,2013,상도로 346-2,-
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,120,201801,24,104500,12,2013,상도로 346-2,-
...,...,...,...,...,...,...,...,...,...,...,...
285394,서울특별시 강남구 삼성동 삼성동힐스테이트 1단지 30,서울특별시 강남구 삼성동,삼성동힐스테이트 1단지,30,202206,20,126000,15,2008,학동로68길 29,중개거래
285395,서울특별시 노원구 공릉동 태강아파트 50,서울특별시 노원구 공릉동,태강아파트,50,202206,26,60000,5,1999,공릉로34길 62,중개거래
285396,서울특별시 노원구 공릉동 태강아파트 60,서울특별시 노원구 공릉동,태강아파트,60,202206,21,70500,4,1999,공릉로34길 62,중개거래
285397,서울특별시 동작구 대방동 보라매센트럴타운 20,서울특별시 동작구 대방동,보라매센트럴타운,20,202206,8,27550,7,2021,여의대방로 200,중개거래


In [4]:
df_c = pd.read_excel('data/database.xlsx')
df_c

,ID,거래금액(만원)_18/1,전세금(만원)_18/1,등락율18/1,전세가율18/1,인접역 수,전용면적,건축년도,Brand,BrandPower,...,인접역 수.17,전용면적.17,건축년도.17,Brand.17,BrandPower.17,구별인구.17,거래량18,cluster18,위도,경도
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,42166.7,4.162633,32.7,0,80,1987.0,0,0,...,0,80,1987.0,0,0,536710,3,4,37.483372,127.053437
1,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,10166.7,16.507998,7.0,1,40,1982.0,0,0,...,1,40,1982.0,0,0,536710,0,4,37.483381,127.057068
2,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,42678.6,0.000000,31.2,1,60,1983.0,0,0,...,1,60,1983.0,0,0,536710,0,4,37.487868,127.068988
3,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,45000.0,0.000000,25.4,1,80,1983.0,0,0,...,1,80,1983.0,0,0,536710,0,4,37.487868,127.068988
4,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,37909.1,9.134234,30.1,1,50,1983.0,0,0,...,1,50,1983.0,0,0,536710,0,4,37.488481,127.072502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,2500.0,6.000000,10.0,1,60,1998.0,0,0,...,1,60,1998.0,0,0,393147,0,2,37.605430,127.082936
4087,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,15000.0,0.000000,42.9,1,80,2005.0,0,0,...,1,80,2005.0,0,0,393147,0,0,37.599447,127.078938
4088,서울특별시 중랑구 중화동 한신아파트 50,31756.2,24571.4,0.000000,77.4,5,50,1997.0,0,0,...,5,50,1997.0,0,0,393147,0,0,37.595961,127.080314
4089,서울특별시 중랑구 중화동 한신아파트 60,36534.6,28111.1,2.996064,76.9,5,60,1997.1,0,0,...,5,60,1997.1,0,0,393147,0,0,37.595961,127.080314


In [11]:
df_b['시'] = df_b['시군구'].str.split(' ', expand=True)[0]
df_b['구'] = df_b['시군구'].str.split(' ', expand=True)[1]
df_b['주소'] = df_b['시'] + ' ' + df_b['구'] + ' ' + df_b['도로명'].astype('str')
df_b.head()

,ID,시군구,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,거래유형,시,구,주소
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,201801,3,130000,4,1987,언주로 3,-,서울특별시,강남구,서울특별시 강남구 언주로 3
1,서울특별시 동작구 신대방동 롯데관악타워 150,서울특별시 동작구 신대방동,롯데관악타워,150,201801,4,74000,27,1996,보라매로5길 51,-,서울특별시,동작구,서울특별시 동작구 보라매로5길 51
2,서울특별시 동작구 신대방동 경남교수 80,서울특별시 동작구 신대방동,경남교수,80,201801,25,65000,17,2001,여의대방로10길 14,-,서울특별시,동작구,서울특별시 동작구 여의대방로10길 14
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,80,201801,27,85800,15,2013,상도로 346-2,-,서울특별시,동작구,서울특별시 동작구 상도로 346-2
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,120,201801,24,104500,12,2013,상도로 346-2,-,서울특별시,동작구,서울특별시 동작구 상도로 346-2


In [12]:
df_bb = df_b[['ID','주소']]
df_bb

,ID,주소
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 언주로 3
1,서울특별시 동작구 신대방동 롯데관악타워 150,서울특별시 동작구 보라매로5길 51
2,서울특별시 동작구 신대방동 경남교수 80,서울특별시 동작구 여의대방로10길 14
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,서울특별시 동작구 상도로 346-2
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,서울특별시 동작구 상도로 346-2
...,...,...
285394,서울특별시 강남구 삼성동 삼성동힐스테이트 1단지 30,서울특별시 강남구 학동로68길 29
285395,서울특별시 노원구 공릉동 태강아파트 50,서울특별시 노원구 공릉로34길 62
285396,서울특별시 노원구 공릉동 태강아파트 60,서울특별시 노원구 공릉로34길 62
285397,서울특별시 동작구 대방동 보라매센트럴타운 20,서울특별시 동작구 여의대방로 200


In [13]:
df_c = pd.merge(df_c, df_bb, left_on='ID', right_on='ID', how='left')
df_c.drop_duplicates(['ID'], inplace=True)
df_c

,ID,거래금액(만원)_18/1,전세금(만원)_18/1,등락율18/1,전세가율18/1,인접역 수,전용면적,건축년도,Brand,BrandPower,...,전용면적.17,건축년도.17,Brand.17,BrandPower.17,구별인구.17,거래량18,cluster18,위도,경도,주소
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,42166.7,4.162633,32.7,0,80,1987.0,0,0,...,80,1987.0,0,0,536710,3,4,37.483372,127.053437,서울특별시 강남구 언주로 3
24,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,10166.7,16.507998,7.0,1,40,1982.0,0,0,...,40,1982.0,0,0,536710,0,4,37.483381,127.057068,서울특별시 강남구 개포로 310
146,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,42678.6,0.000000,31.2,1,60,1983.0,0,0,...,60,1983.0,0,0,536710,0,4,37.487868,127.068988,서울특별시 강남구 삼성로4길 17
171,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,45000.0,0.000000,25.4,1,80,1983.0,0,0,...,80,1983.0,0,0,536710,0,4,37.487868,127.068988,서울특별시 강남구 삼성로4길 17
199,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,37909.1,9.134234,30.1,1,50,1983.0,0,0,...,50,1983.0,0,0,536710,0,4,37.488481,127.072502,서울특별시 강남구 개포로 516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214262,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,2500.0,6.000000,10.0,1,60,1998.0,0,0,...,60,1998.0,0,0,393147,0,2,37.605430,127.082936,서울특별시 중랑구 동일로140길 88
214281,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,15000.0,0.000000,42.9,1,80,2005.0,0,0,...,80,2005.0,0,0,393147,0,0,37.599447,127.078938,서울특별시 중랑구 동일로129길 35
214302,서울특별시 중랑구 중화동 한신아파트 50,31756.2,24571.4,0.000000,77.4,5,50,1997.0,0,0,...,50,1997.0,0,0,393147,0,0,37.595961,127.080314,서울특별시 중랑구 동일로 752
214385,서울특별시 중랑구 중화동 한신아파트 60,36534.6,28111.1,2.996064,76.9,5,60,1997.1,0,0,...,60,1997.1,0,0,393147,0,0,37.595961,127.080314,서울특별시 중랑구 동일로 752


In [14]:
df_c.to_excel('data/주소가 있는 데이터 프레임.xlsx')

In [15]:
df = pd.merge(df_c, df_a, how='left', left_on='주소', right_on='도로명주소')
df

,ID,거래금액(만원)_18/1,전세금(만원)_18/1,등락율18/1,전세가율18/1,인접역 수,전용면적,건축년도,Brand,BrandPower,...,지역난방TF,복도식TF,혼합형TF,계단식TF,메이저시공사TF,조합시행TF,철근철골구조TF,홈네트워크TF,관리시설수,부대복리시설합
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,42166.7,4.162633,32.7,0,80,1987.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,10166.7,16.507998,7.0,1,40,1982.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,42678.6,0.000000,31.2,1,60,1983.0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
3,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,45000.0,0.000000,25.4,1,80,1983.0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
4,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,37909.1,9.134234,30.1,1,50,1983.0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,"관리사무소, 노인정, 어린이놀이터"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,2500.0,6.000000,10.0,1,60,1998.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4295,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,15000.0,0.000000,42.9,1,80,2005.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4296,서울특별시 중랑구 중화동 한신아파트 50,31756.2,24571.4,0.000000,77.4,5,50,1997.0,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,5.0,"관리사무소, 노인정, 보육시설, 어린이놀이터, 유치원"
4297,서울특별시 중랑구 중화동 한신아파트 60,36534.6,28111.1,2.996064,76.9,5,60,1997.1,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,5.0,"관리사무소, 노인정, 보육시설, 어린이놀이터, 유치원"


In [16]:
df.to_excel('data/test.xlsx')